# Audit
by David Vicente Martínez & Daniel Loscos Barroso

## 1. Bulding the dataset

The first thing we found out about the database was that the data and the headers where stored in two different files, we merged them in to the file called "o-database".

The dataset consisted of different mushrooms classified as edible or poisonous and their characteristics. 8124 instances were recorded each with information about 22 attributes, all of them nominally valued. Across the database 2480 attribute values where missing.

In [3]:
from __future__ import print_function

import pandas as pd
pd.__version__

mushroom_dataframe = pd.read_table("o-database", sep=",")
mushroom_dataframe.describe()

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-bellow-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148
